In [2]:
import json
import os
import pandas as pd
import time

from dotenv import load_dotenv
from openrouteservice import client, directions, isochrones, places

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [14]:
output_dir = '../data/curated/'

## set variables
load_dotenv()
tokens = []
for i in range(1,24): ## number of keys
    token = os.environ.get(f'token_{i}')
    tokens.append(token)
    
melbourne_cbd = [144.9631, -37.8136] # taken from google maps 
cbd_dict = {}

In [15]:
## read apartment data
property_data = json.load(open( f"{output_dir}cbd_distance.json" ))
cbd_dict = property_data['CBD_distance']

completed = {k:v for k,v in cbd_dict.items() if v is not None}
len(completed)

7686

In [6]:
# load previously run cbd_dict
cbd_dict = json.load(open(f"{output_dir}cbd_distance.json"))

In [88]:
token = tokens[15] #do with more tokens 

## set x and y
x = len(completed)
y = len(completed)+500 ## from smallest API quota
if y > len(property_data['Coordinates'].keys()):
    y = len(property_data['Coordinates'].keys())

for index in range(x,y):
    backwards = list(map(float,property_data['Coordinates'][str(index)][1:-1].split(',')))
    coords = [[backwards[1], backwards[0]]]
    ors = client.Client(key=token)
    
    all = [[str(coords[0][0]),str(coords[0][1])],[str(melbourne_cbd[0]),str(melbourne_cbd[1])]]

    ## get car route
    request = {'coordinates': all,
            'profile': 'driving-car',
            'geometry': 'true',
            'format_out':'geojson'}
    route = ors.directions(**request)
    info = route['features'][0]['properties']['summary']

    ## add information
    cbd_dict[str(index)] = info
    time.sleep(0.5)

In [89]:
# check how many cbd distances have been filled in the property data 
completed = {k:v for k,v in cbd_dict.items() if v is not None}
print(len(completed))

13226


Driveways are longer than 350m to the main road and cannot be detected by ors. Hence, impute as 0

In [44]:
# imputing 0 due to 'Could not find routable point within a radius of 350.0 meters of specified coordinate' error
# the driveway is 450m not on map, api limited to 350m to search for a route
cbd_dict[str(10561)] = 0 

In [64]:
# imputing 0 due to 'Could not find routable point within a radius of 350.0 meters of specified coordinate' error
# the driveway is 500m not on map, api limited to 350m to search for a route
cbd_dict[str(11129)] = 0 

Not on map

In [75]:
# imputing 0 due to 'Could not find routable point within a radius of 350.0 meters of specified coordinate' error
# Benalla residence not forming route
cbd_dict[str(11728)] = 0 
cbd_dict[str(11729)] = 0

Road closed in Rockbank 

In [38]:
# imputing 0 due to 'Could not find routable point within a radius of 350.0 meters of specified coordinate' error
# road closed
cbd_dict[str(10497)] = 0
cbd_dict[str(10581)] = 0
cbd_dict[str(10590)] = 0
cbd_dict[str(10591)] = 0


House on the ocean?

In [86]:
# imputing 0 due to 'Could not find routable point within a radius of 350.0 meters of specified coordinate' error
# house is on the water it seems
cbd_dict[str(12797)] = 0

Save data

In [90]:
property_data['CBD_distance'] = cbd_dict

## write json
json.dump(property_data, open(f"{output_dir}cbd_distance.json", 'w'))

# write csv
data = pd.read_json(f"{output_dir}cbd_distance.json")
data.to_csv(f"{output_dir}cbd_distance.csv")